In [2]:
import sys
sys.path.append('../../../../')

In [3]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

2025-04-15 14:36:36.637279: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-15 14:36:36.751776: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-15 14:36:41.820242: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/iscb/wolfson/doririmon/anaconda3/

In [4]:
from utils import load_as_pickle

/home/iscb/wolfson/doririmon/anaconda3/envs/ubinet-gpu/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [5]:
import sys
import os
from typing import List
from data_preparation.patch_to_score.v1.create_protein_objects.utils import create_patches_dict, create_90_percentile
from data_preparation.ScanNet.db_creation_scanNet_utils import save_as_pickle, load_as_pickle


def create_merged_protein_object_dict(dir_path):
    indexes = load_as_pickle(os.path.join(dir_path, 'indexes.pkl'))
    merged_dict = {}
    for i in range(len(indexes) - 1):
        d = load_as_pickle(os.path.join(
            dir_path, 'proteinObjectsWithEvoluion' + str(i)))
        for key, value in d.items():
            merged_dict[key] = value
    return merged_dict


def create_paths(*paths: List[str]) -> None:
    for path in paths:
        os.makedirs(path, exist_ok=True)


def main(all_predictions_path: str, save_dir_path: str, with_pesto: bool):
    """
    Create protein objects.

   :param str all_predictions_path: path to a pickle file containing all predictions in the form of a dictionary
   :param str save_dir_path: path to a directory where the protein objects will be saved
   :param bool with_pesto: whether to all_predictions object includes pesto predictions or not
   :return: a list of protein objects
   :rtype: List[Protein]
    """
    create_paths(save_dir_path)
    percentile_90_path = os.path.join(save_dir_path, 'percentile_90.pkl')
    merged_dict_path = os.path.join(
        save_dir_path, 'merged_protein_objects.pkl')

    all_predictions = load_as_pickle(all_predictions_path)

    if not os.path.exists(percentile_90_path):
        percentile_90 = create_90_percentile(all_predictions)
        save_as_pickle(percentile_90, os.path.join(
            save_dir_path, 'percentile_90.pkl'))
    else:
        percentile_90 = load_as_pickle(percentile_90_path)

    # CREATING THE PATCHES IN BATCHES OF 1500 PROTEINS. SEE SCRIPTS/RUN_DATA_DEVELOPMENT.SH
    # WE CAN RUN THIS ON CPU'S (CAN DO MULTIPLE AT A TIME)
    i = int(sys.argv[1])  # TODO
    plddt_threshold = 50
    create_patches_dict(i, save_dir_path, plddt_threshold, all_predictions, percentile_90, with_pesto)

    if not os.path.exists(merged_dict_path):
        merged_dict = create_merged_protein_object_dict(save_dir_path)
        save_as_pickle(merged_dict, merged_dict_path)
    else:
        merged_dict = load_as_pickle(merged_dict_path)


In [6]:
all_predictions_path = '/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/results/ScanNet/all_predictions_0304_MSA_True_with_pesto.pkl'
all_predictions = load_as_pickle(all_predictions_path)

In [7]:
percentile_90 = create_90_percentile(all_predictions)

percentile_90: 0.26495680809021005


In [11]:
all_predictions.keys()

dict_keys(['dict_resids', 'dict_sequences', 'dict_sources', 'dict_pdb_files', 'dict_predictions_ubiquitin', 'dict_predictions_interface', 'pesto_protein', 'pesto_dna_rna', 'pesto_ion', 'pesto_ligand', 'pesto_lipid'])

In [18]:
all_predictions['dict_resids']['A0A022QL98']

array([['0', 'A', '1'],
       ['0', 'A', '2'],
       ['0', 'A', '3'],
       ...,
       ['0', 'A', '987'],
       ['0', 'A', '988'],
       ['0', 'A', '989']], dtype='<U21')

In [2]:
len(all_predictions)

NameError: name 'all_predictions' is not defined